<h1><span style="color:red">NEMO Annotation for SuAVE</span></h1>

This notebook uses the NEMO (Named Entities Made Obvious) service developed at Microsoft Research (author: Dr. Silviu Cuzerzan, see https://doi.org/10.1145/2633211.2634360) to generate a set of named entities for user's text


## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

## 2. Setting up the environment, importing libraries, defining functions

In [ ]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

pn.extension()
def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint


# specific imports
import requests
import re
import nemofunc as nemo


In [ ]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

## 3. Select a survey file from SuAVE or import a local CSV file

In [ ]:
data_select = pn.widgets.RadioBoxGroup(name='Select notebook', options=['Load survey file from SuAVE', 
                                                                        'Import a local CSV file'], 
                                       inline=False)
data_select

In [ ]:
data_input = pn.widgets.FileInput()
    
def check_selection():
    if data_select.value == 'Load survey file from SuAVE':
        global fname
        fname = absolutePath + csv_file
        printmd("<b><span style='color:red'>SuAVE survey will be loaded. Continue to step 4.</span></b>")

    else:
        message = pn.pane.HTML("<b><span style='color:red'>Upload data and continue to step 4.</span></b>")
        return pn.Column(message, data_input)
    
check_selection()

## 4. Visualize the data and select a text variable to parse

In [ ]:
if not pd.isnull(data_input.filename):
    fname = absolutePath + data_input.filename
    data_input.save(fname)

df = panellibs.extract_data(fname)
panellibs.slider(df)


## 5. Generate pre-defined #multi variables for entity types extracted by NEMO

In [ ]:
varcols = df.columns.tolist()
# remove any variable names are unlikely to contain parsable text 
varcols = [x for x in varcols if '#number' not in x and '#date' not in x and '#img' not in x and '#href' not in x and '#link' not in x]

# Left panel
left_text = pn.Row("####Select Variables for NEMO Processing", margin=(0,0,-15,270))
binary_selector = pn.widgets.CrossSelector(options=varcols, width=630)
left_panel = pn.Column(left_text, binary_selector, css_classes=['widget-box'], margin=(0,30,0,0))

remap_text = pn.pane.Markdown('####      Make selections and run the next cell ', width=650)

# Display widgets
widgets = pn.Row(left_panel)
full_display = pn.Column(widgets,remap_text)
full_display

In [ ]:

# Replace NA with empty in each row
# Convert row to string
# Join row with spaces
concatted = df[binary_selector.value].fillna('').astype(str).dropna().apply(lambda row: ' '.join(row), axis=1)


## 6. Run NEMO

In [ ]:
def extract_nemo(payload):
#     print(payload)
    df, respjson = nemo.nemo_annotate(payload)
    return df
extracted_df = concatted.progress_apply(extract_nemo)

## 7. Add generated tags to dataframe and explore

In [ ]:
extracted_multi_df=pd.DataFrame()
import numpy as np
nan = np.nan
rows=[]
for i in concatted.index:
    print ("###### index == " + str(i))
    rows.append(nemo.create_nemo_dict(extracted_df[i]))

extracted_multi_df = pd.DataFrame(rows)


In [ ]:
# explore the generated dataframe
panellibs.slider(extracted_multi_df)

## 8. Reoder the columns, and decide which columns to keep

In [ ]:
# re-order the columns in the df
columnTitles = nemo.column_order()
extracted_multi_df = extracted_multi_df.reindex(columns=columnTitles)

In [ ]:
tags_to_keep = pn.widgets.RadioBoxGroup(name='Tags to keep:', options=['Pre-modeled categories: Location, Geopolitical, Organiation, Person', 
                                                                        'All internal NEMO output'], 
                                       inline=False)
tags_to_keep

#### Columns to keep:

        "e_G-y": "Entity: Geopolitical#multi#sortquan",
        "e_G-y_WP": "Entity: Geopolitical URL#hidden",
        "e_O-y": "Entity: Organization#multi#sortquan",
        "e_O-y_WP": "Entity: Organization URL#hidden",
        "e_L-y": "Entity: Location#multi#sortquan",
        "e_L-y_WP": "Entity: Location URL#hidden",
        "e_P-y": "Entity: Personal#multi#sortquan",
        "e_P-y_WP": "Entity: Personal URL#hidden",

        "e_C-y": "Entity: Abstract#multi#sortquan",
        "e_C-y_WP" : "Entity: Abstract URL#hidden",


In [ ]:
cols_to_keep = [
     "e_G-y",
    "e_G-y_WP",
    "e_O-y",
    "e_O-y_WP",
    "e_L-y",
    "e_L-y_WP",
    "e_P-y",
    "e_P-y_WP",

    "e_C-y",
    "e_C-y_WP"   
]

if tags_to_keep.value == 'Pre-modeled categories: Location, Geopolitical, Organiation, Person':
    df.drop(df.columns.difference(['a','b']), 1, inplace=True)



## 9. Rename the columns

In [ ]:
# this is a complete list of columns to rename
columns_dict = nemo.columns_dict()


In [ ]:
#rename the columns
extracted_multi_df_ren = extracted_multi_df.rename(columns=dict(columns_dict))

In [ ]:
df_new = pd.concat([df, extracted_multi_df_ren], axis=1)
print('Dimensions:\n --- The original df: ' +str(df.shape) +'\n --- The NEMO-generated df: '+ str(extracted_multi_df.shape)+'\n --- The concatenated df:' +str(df_new.shape))


### Only keep columns with more than N records as facets (for new fields, or for all fields), make others #hidden

In [ ]:
# set min number of values in a column
mincount = 20

# whether to drop columns from the original dataframe as well
drop_from_original=True

if drop_from_original:
    start_column = 0
else:
    start_column = len(df.columns)

for column in df_new.columns:
    if (df_new.columns.get_loc(column) > start_column) & (df_new[column].count() < mincount):
#         print("Dropping: " + column +" at location " + str(df_new.columns.get_loc(column)) + " with " +str(df_new[column].count()) +" values.")
        if column.find("#multi") > 0:
            new_col = column.split('#')[0]+"#multi#hidden"
        elif column.find("#link") > 0:
            new_col = column
        else:
            new_col = column.split('#')[0]+"#hidden"

        df_new = df_new.rename(columns={column:new_col})
        print(column + " renamed to "+ new_col)
#         df_new.drop([column],axis=1)
            


## 10. Visualize the generated dataframe

In [ ]:
panellibs.slider(df_new)

In [ ]:
# if needed, save to a local file. OPTIONAL
df_to_save = df_new.copy().fillna('')

df_to_save.to_csv('ecrr_after_nemo.csv', index=None)


## 11. Save the new version of CSV file, and give a name to new survey

In [ ]:
# new filename

if data_select.value == 'Import a local CSV file':
    csv_file = data_input.filename

new_file = absolutePath + csv_file[:-4]+'_v1.csv'
printmd("<b><span style='color:red'>A new temporary file will be created at: </span></b>")
print(new_file)
df_to_save.to_csv(new_file, index=None)

In [ ]:
#Input survey name
import ipywidgets as widgets
from IPython.display import display

input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)

In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

In [ ]:
url = 'http://suave2.sdsc.edu/getSurveyDzc?user='+user+'&file='+csv_file[len(user)+1:-4]
url

In [ ]:
# read the current survey file, if this is suave2
# example: http://suave2.sdsc.edu/getSurveyDzc?user=zaslavsk&file=Alianza_2_5_21

import requests
import json
r=requests.get(url).json()
views = str(r['views'])
views

## 12. Generate the survey and create survey URL

In [ ]:

referer = survey_url.split("/main")[0] +"/"
upload_url = referer + "uploadCSV"
new_survey_url_base = survey_url.split(user)[0]

import requests
import re
csv = {"file": open(new_file, "rb")}

if data_select.value == 'Import a local CSV file':
    dzc_file = ''
    views = '1110001'
    view='grid'

upload_data = {
    'name': input_text.value,
    'dzc': dzc_file,
    'user':user
}
headers = {
    'User-Agent': 'suave user agent',
    'referer': referer
}

r = requests.post(upload_url, files=csv, data=upload_data, headers=headers)

if r.status_code == 200:
    printmd("<b><span style='color:red'>New survey created successfully</span></b>")
    regex = re.compile('[^0-9a-zA-Z_]')
    s_url = survey_name
    s_url =  regex.sub('_', s_url)

    url = new_survey_url_base + user + "_" + s_url + ".csv" + "&views=" + views + "&view=" + view
    print(url)
    printmd("<b><span style='color:red'>Click the URL to open the new survey</span></b>")
else:
    printmd("<b><span style='color:red'>Error creating new survey. Check if a survey with this name already exists.</span></b>")
    printmd("<b><span style='color:red'>Reason: </span></b>"+ str(r.status_code) + " " + r.reason)